In [ ]:
import serial
from tlm_parser import Parser
import struct

In [ ]:
IMUMAG_DATA = '<Lhhhhhhhhhh'
BARO_DATA = '<LLhH'
GNSS_DATA = '<LLHBBllllllHHH'
FDR_DATA = '<LLL'

def parse_packet(flags, payload):
    #print('%02x' % flags, payload.hex())
    
    if flags & Parser.FLAG_IMUMAG:
        vals = struct.unpack(IMUMAG_DATA, payload)
        payload = payload[struct.calcsize(IMUMAG_DATA):]
        print((vals[0], 'IMUMag', ) + vals[1:])
        
    if flags & Parser.FLAG_BARO:
        vals = struct.unpack(BARO_DATA, payload)
        payload = payload[struct.calcsize(BARO_DATA):]
        print((vals[0], 'Baro', ) + vals[1:])
        
    if flags & Parser.FLAG_GNSS:
        vals = struct.unpack(GNSS_DATA, payload)
        payload = payload[struct.calcsize(GNSS_DATA):]
        print((vals[0], 'Gnss', ) + vals[1:])
    
    if flags & Parser.FLAG_FDR:
        vals = struct.unpack(FDR_DATA, payload)
        payload = payload[struct.calcsize(FDR_DATA):]
        print((vals[0], 'FDR', ) + vals[1:])
        
    if len(payload):
        print(payload.hex())

In [ ]:
def radio_open(port: str, baud: int):
    return serial.Serial(port, baudrate=baud, timeout=1)
    
def radio_loop(ser) -> None:
    # Initialize serial connection
    parser = Parser(parse_packet)
    stopping = False
    while not stopping:
        try:
            buf = ser.read(ser.in_waiting)
            if len(buf): parser.parse_bytes(buf)
        except KeyboardInterrupt:
            stopping = True
        except:
            ser.close()
    print('Exiting...')
        

In [ ]:
radio = radio_open('COM3', 9600)
radio_loop(radio)
radio.close()

In [ ]:
radio.close()